In [1]:
# !/usr/bin/env python
# coding: utf-8

from boiles.config import OC, OP
from boiles.solvers.solver_config import SC
from boiles.test_cases.sod_60 import SodDisper60
import os
import shutil

In [2]:
case_num = 1
OC.seed = 100
OC.initial_samples = 50
OC.opt_iterations = 50
OC.opt_bounds = [(1, 20), (1, 200), (40, 90)]
OC.fun_bounds = [(1, 20), (1, 200), (0.4, 0.9)]
OC.increment = (1, 1, 0.01)
OC.dim_inputs = 3
OC.dim_outputs = 1
OC.discrete = {
        "activate": False,
        "method": "constrained",  # selection from "tree" and "constrained"
        "plot": True,
        "max_level": 6,  # only valid if method is tree.
        "counter": 1
    }

OP.test_cases = [SodDisper60]
OP.ref_point = [SodDisper60.ref_point]

SC.alpaca_executable = "/media/yiqi/Fengyiqi/TUM/TENO5/teno5_eta_sod_conference/ALPACA_60_TENO5RL"

if os.path.exists("log.txt"):
    os.remove("log.txt")
if os.path.exists("discrete"):
    shutil.rmtree("discrete")
os.makedirs("discrete")

In [3]:
from ax import ChoiceParameter, SearchSpace, ParameterType, SimpleExperiment, Data

import torch
from ax.metrics.noisy_function import NoisyFunctionMetric
from ax.service.utils.report_utils import exp_to_df
from ax.runners.synthetic import SyntheticRunner
from boiles.models.factory import get_moo_ehvi_botorch, get_botorch

import numpy as np

from boiles.utils import log

if not os.path.exists('runtime_data'):
    os.makedirs('runtime_data')

In [4]:
"""
is_ordered: If False, the parameter is a categorical variable.
            Defaults to False if parameter_type is STRING and ``values``
            is longer than 2, else True.
sort_values: Whether to sort ``values`` before encoding.
             Defaults to False if ``parameter_type`` is STRING, else
             True.

"""
search_space = SearchSpace(
    parameters=[
        ChoiceParameter(
            name="q",
            parameter_type=ParameterType.INT,
            values=torch.arange(OC.opt_bounds[0][0], OC.opt_bounds[0][1] + 1),
            is_ordered=True,
            sort_values=True
        ),
        ChoiceParameter(
            name="cq",
            parameter_type=ParameterType.INT,
            values=torch.arange(OC.opt_bounds[1][0], OC.opt_bounds[1][1] + 1),
            is_ordered=True,
            sort_values=True
        ),
        ChoiceParameter(
            name="eta",
            parameter_type=ParameterType.INT,
            values=torch.arange(OC.opt_bounds[2][0], OC.opt_bounds[2][1] + 1),
            is_ordered=True,
            sort_values=True
        ),
    ]
)

In [5]:
#!/usr/bin/env python3
import xml.etree.ElementTree as ET
from boiles.objective.sodshocktube import Sod
from botorch.test_functions.base import MultiObjectiveTestProblem
from boiles.utils import *
from boiles.test_cases.sod_60 import SodDisper60
import time

schemefile = "/media/yiqi/Fengyiqi/TUM/TENO5/teno5_eta_sod_conference/runtime_data/scheme.xml"
inputfile = "/media/yiqi/Fengyiqi/TUM/TENO5/teno5_eta_sod_conference/runtime_data/sod_60.xml"

def configure_scheme_xml(action):
    tree = ET.ElementTree(file=schemefile)
    root = tree.getroot()
    root[0].text = "0"
    q, cq, eta = action[0], action[1], action[2]
    ct = 1e-5
    d1, d2 = np.round((2 + eta) / 4, 4), np.round((1 - eta) / 2, 4)

    for i, para in enumerate([q, cq, d1, d2, ct]):
        root[i + 2].text = str(para)

    tree.write(schemefile)


class SodMetrics(MultiObjectiveTestProblem):
    r"""Two objective problem composed of the Shu-Osher problem and Taylor-Green Vortex.

    ShuOsher:

        maximize gradient near jump but minimize gradient difference along continuous region

    TGV:

        log-scale difference between kinetic energy ans -5/3 principle

    """
    dim = OC.dim_inputs
    num_objectives = OC.dim_outputs
    _bounds = OC.opt_bounds
    _ref_point = OP.ref_point
    
    def __init__(self, case_num, noise_std: float = None, negate: bool = False) -> None:
        r"""Constructor for ShuOsher-TGV.

        Arguments:
            noise_std: Standard deviation of the observation noise.
            negate: If True, negate the objectives.
        """

        super().__init__(noise_std=noise_std, negate=negate)
        self.sod_case_num = case_num  # count the number of simulated shu case
        self.sod_disper_sum_dic = {}

    def evaluate_true(self, X):
        r"""
        An abstract method of the base class
        """
        return NotImplemented


    def sod_error(self, x):
        
        q = x["q"]
        cq = x["cq"]
        eta = x["eta"] / 100
        # print(f"-----------\n{self.sod_disper_dic}\n-----------------")
        # find if the result based on current cq and q has been evaluated.
        if self.sod_disper_sum_dic.get(f"{str(q)}, {str(cq)}, {format(eta, '.2f')}", None) is None:
            # when debugging, uses an easy to evaluate function, else, uses the Shu-Osher problem.
            sod_disper = sod_disper_sum_error([q, cq, eta], self.sod_case_num)
            if OC.discrete["activate"]:
                append_to_csv(f"discrete/data.csv", [x["q"], x["cq"], x["eta"], sod_disper])

            self.sod_disper_sum_dic[f"{str(q)}, {str(cq)}, {format(eta, '.2f')}"] = sod_disper
            self.sod_case_num += 1
        else:
            sod_disper = self.sod_disper_sum_dic[f"{str(q)}, {str(cq)}, {format(eta, '.2f')}"]

        return sod_disper



def sod_disper_sum_error(para, case_num):
    r"""
    Run Shu-Osher problem simulation and get dispersion error or shock capturing indicator.

    Two objectives can be directly obtained from a csv data file which is obtained from
    previously simulated results by fixing the random seed, which could save a lot of time.

    :param cq: cq in WENO5-CU6-M1
    :param q: q in WENO5-CU6-M1
    :param case_num: case number.
    :return: dispersion error and shock capturing indicator of Shu-Osher problem.
    """

    q, cq, eta = para
    log(f'Start Sod case.{case_num:<2} with q={q:<4} cq={cq:<4} and eta={eta:<4}')
    configure_scheme_xml(para)
#     alpaca.build_workfolder(case_num)
#     alpaca.run_alpaca(case_num, cpu_num=OP.test_cases[0].cpu_num, inputfile_name=OP.test_cases[0].inputfile)
    os.system(f"cd runtime_data; mpiexec -n 1 {SC.alpaca_executable} {inputfile}")
    os.system(f"mv runtime_data/sod_60 runtime_data/case.{case_num}")
    
#     results_folder = os.path.join(OC.case_folder, f"case.{str(case_num)}/{OP.test_cases[0].inputfile[:-4]}/domain")
    sod = Sod(
        results_folder=f"runtime_data/case.{case_num}/domain",
        result_filename="data_0.200*.h5",
        git=False,
        plot=False
    )
    sod_disper, sod_disper_raw, state_disper = sod.objective_sum_disper()

    log(f"Dispersion error: {format(sod_disper, '.4f')}")

    append_to_csv(f'runtime_data/sod_runtime_samples.csv',
                  [case_num, q, cq, eta, sod_disper, sod_disper_raw, state_disper])
    # print(type(shu_disper), type(shu_shock))
    # print(shu_disper, shu_shock)
    return sod_disper

sod_metrics = SodMetrics(negate=False, case_num=case_num).to(
    dtype=torch.double,
    device=torch.device("cpu"),
)

def sod_disper_sum_error_(x: np.ndarray):
    return {"disper": (sod_metrics.sod_error(x), 0)}

In [6]:
experiment = SimpleExperiment(
    name="teno5_opt",
    search_space=search_space,
    evaluation_function=sod_disper_sum_error_,
    objective_name='disper',
    minimize=True,
)

In [7]:
from ax.modelbridge import get_sobol

sobol = get_sobol(experiment.search_space, seed=OC.seed)
experiment.new_batch_trial(generator_run=sobol.gen(OC.initial_samples))

BatchTrial(experiment_name='teno5_opt', index=0, status=TrialStatus.CANDIDATE)

In [8]:
import warnings
from boiles.utils import save_single_model_state
from boiles.models.botorch_defaults import get_and_fit_model

warnings.filterwarnings('ignore', category=UserWarning)
os.system("rm -rf runtime_data/case.* runtime_data/opt_history.csv runtime_data/model_state runtime_data/sod_runtime_samples.csv")

0

In [9]:
start = 0
iteration = OC.opt_iterations

# used to generate the initial model
ei_model = get_botorch(
    experiment=experiment,
    data=experiment.eval(),
    model_constructor=get_and_fit_model,
    device=torch.device('cpu'),
)
# print(ei_model.model.model)
# plot_runtime_contour(['sod', 'tgv'], ehvi_model, 0)
# runtime_cross_validation(ehvi_model, 0)

Start Sod case.1  with q=5    cq=136  and eta=0.79
Dispersion error: 0.3459
Start Sod case.2  with q=13   cq=68   and eta=0.45
Dispersion error: 0.0661
Start Sod case.3  with q=18   cq=167  and eta=0.76
Dispersion error: 0.1694
Start Sod case.4  with q=10   cq=35   and eta=0.59
Dispersion error: 0.0590
Start Sod case.5  with q=7    cq=177  and eta=0.51
Dispersion error: 0.2451
Start Sod case.6  with q=17   cq=22   and eta=0.85
Dispersion error: 0.1276
Start Sod case.7  with q=12   cq=122  and eta=0.54
Dispersion error: 0.1461
Start Sod case.8  with q=1    cq=78   and eta=0.7 
Dispersion error: 0.8941
Start Sod case.9  with q=3    cq=153  and eta=0.56
Dispersion error: 0.3023
Start Sod case.10 with q=11   cq=46   and eta=0.68
Dispersion error: 0.1379
Start Sod case.11 with q=16   cq=146  and eta=0.49
Dispersion error: 0.1527
Start Sod case.12 with q=8    cq=54   and eta=0.88
Dispersion error: 0.4088
Start Sod case.13 with q=9    cq=111  and eta=0.75
Dispersion error: 0.3079
Start Sod ca

In [10]:
save_single_model_state(ei_model, 0)

for i in range(start, int(start) + int(iteration)):
    log(f"Start {i + 1} iteration...")
    ei_model = get_botorch(
        experiment=experiment,
        data=experiment.eval(),
        model_constructor=get_and_fit_model,
        device=torch.device('cpu'),
    )

    trial = experiment.new_trial(generator_run=ei_model.gen(1))
    ei_data = Data.from_multiple_data([experiment.eval(), trial.fetch_data()])
    exp_df = exp_to_df(experiment)
    exp_df.to_csv(f'{OC.case_folder}/opt_history.csv')

    # plot_runtime_contour(['sod', 'tgv'], ehvi_model, i+1)
    # runtime_cross_validation(ehvi_model, i+1)
    save_single_model_state(ei_model, i + 1)
    # cross_validation(ehvi_model, i)
    if OC.discrete["activate"]:
        OC.discrete["counter"] += 1

    # write_to_csv(f'{OC.case_folder}/hv_list.csv', ehvi_hv_list.reshape(-1, 1))

Start 1 iteration...
Start Sod case.51 with q=19   cq=32   and eta=0.57
Dispersion error: 0.0399
Start 2 iteration...
Start Sod case.52 with q=18   cq=58   and eta=0.58
Dispersion error: 0.0456
Start 3 iteration...
Start Sod case.53 with q=19   cq=52   and eta=0.55
Dispersion error: 0.0814
Start 4 iteration...
Start Sod case.54 with q=19   cq=1    and eta=0.55
Dispersion error: 0.1360
Start 5 iteration...
Start Sod case.55 with q=19   cq=54   and eta=0.54
Dispersion error: 0.0421
Start 6 iteration...
Start 7 iteration...
Start Sod case.56 with q=19   cq=55   and eta=0.54
Dispersion error: 0.0425
Start 8 iteration...
Start Sod case.57 with q=16   cq=1    and eta=0.85
Dispersion error: 0.1731
Start 9 iteration...
Start Sod case.58 with q=16   cq=1    and eta=0.4 
Dispersion error: 0.1092
Start 10 iteration...
Start 11 iteration...
Start Sod case.59 with q=19   cq=112  and eta=0.43
Dispersion error: 0.0442
Start 12 iteration...
Start Sod case.60 with q=14   cq=200  and eta=0.4 
Dispersion

In [11]:
sod_opt = Sod(results_folder="runtime_data/case.73/domain")
sod_lin = Sod(results_folder="runtime_data/sod_lin/domain")
sod_opt.objective_sum_disper(), sod_lin.objective_sum_disper(), sod_opt.result["kinetic_energy"].sum(), sod_lin.result["kinetic_energy"].sum()

((0.019205414676548956, 0.019205414676548956, 'completed'),
 (0.04012208174739642, 0.04012208174739642, 'completed'),
 4.27111806122469,
 4.267233218013098)

In [32]:
configure_scheme_xml((16, 22, 0.3))